In [0]:
bronze_df=spark.sql("SELECT * FROM datamodeling.bronze.bronze_source")
display(bronze_df)

In [0]:
from pyspark.sql.functions import col

silver_df=bronze_df.filter(col("show_id").isNotNull()&col("title").isNotNull()&col("director").isNotNull()&col("cast").isNotNull()&col("release_year").isNotNull()&col("rating").isNotNull()&col("country").isNotNull()&col("date_added").isNotNull()&col("description").isNotNull())
display(silver_df)

In [0]:
# directors_df=silver_df.filter((col("director").isNotNull()) & (col("director") != 'unknown'))
# display(directors_df)

In [0]:
# from pyspark.sql.functions import count

# director_counts = directors_df.groupBy("director").agg(count("*").alias("num_titles")).orderBy(col("num_titles").desc())
# display(director_counts)


In [0]:
# display(silver_df.filter(col("release_year")>2020))

In [0]:
from pyspark.sql.functions import split, col,explode, trim

silver_df = silver_df.withColumn("country_array", split(col("country"), ","))

country_exploded_df = silver_df.withColumn("country", explode(col("country_array"))).withColumn("country", trim(col("country"))).drop("country_array")
display(country_exploded_df)



In [0]:
from pyspark.sql.functions import when, regexp_extract, col

silver_df2 =country_exploded_df.withColumn("duration_minutes",when(col("type") == "Movie", regexp_extract(col("duration"), r"(\d+)", 1))).withColumn("duration_seasons",when(col("type") == "TV Show", regexp_extract(col("duration"), r"(\d+)", 1))).fillna({"duration_minutes":"0","duration_seasons":0}).drop("duration")
display(silver_df2)


In [0]:
from pyspark.sql.functions import to_date, trim, when, col, lit

df_clean = silver_df2.withColumn(
    "date_added_clean",
    when(
        to_date(trim(col("date_added")), "MMMM d, yyyy").isNotNull(),
        trim(col("date_added"))
    ).otherwise(lit("Unknown"))
)
display(df_clean)

In [0]:
from pyspark.sql.functions import when, col, trim
df_silver = df_clean .withColumn(
    "country",
    when((trim(col("country")) == "") | (col("country").isNull()), "Unknown")
    .otherwise(col("country"))
)
df_silver2 = df_silver.withColumn(
    "rating",
    when(col("rating").rlike("(?i)min"), "Not Rated")  # if contains "min" or "Min"
    .otherwise(col("rating"))
)
display(df_silver2)

In [0]:
df_silver2.write.format("delta").mode("overwrite").saveAsTable("datamodeling.silver.silver_source")


In [0]:
display(spark.sql("SELECT * FROM datamodeling.silver.silver_source"))